In [1]:
import pandas as pd
import altair as alt

In [2]:
# Load the dataset
file_path = '/Users/shirleyaraizasantaella/Documents/UMich/Grad School Year 3/SI 649 Info Visualization/final project/SI649---Final-Project-Lizett-Shirley-Humaira/2010_service_interruptions.csv'  # Replace with your CSV file path
interruptions_2010 = pd.read_csv(file_path)

### Trend of water shutoffs over time

In [3]:
interruptions_2010['WO_CREATEDATE'] = pd.to_datetime(interruptions_2010['WO_CREATEDATE'])
interruptions_2010['YearMonth'] = interruptions_2010['WO_CREATEDATE'].dt.to_period('M').dt.to_timestamp()

shutoffs_over_time = interruptions_2010.groupby('YearMonth').size().reset_index(name='Shutoff_Count')

# Create the line chart
alt.Chart(shutoffs_over_time).mark_line().encode(
    x=alt.X('YearMonth:T', title='Year-Month'),
    y=alt.Y('Shutoff_Count:Q', title='Number of Shutoffs'),
    tooltip=['YearMonth:T', 'Shutoff_Count:Q']
).properties(
    title='Trend of Water Shutoffs Over Time',
    width=600,
    height=400
)

/var/folders/6t/t91z119x51q_lpb_lspp0sjw0000gn/T/ipykernel_74577/860917799.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  interruptions_2010['WO_CREATEDATE'] = pd.to_datetime(interruptions_2010['WO_CREATEDATE'])


alt.Chart(...)

### Geographic Distribution of Shutoffs

In [4]:
import pandas as pd
import altair as alt
import requests
import json

# Disable the row limit for Altair
alt.data_transformers.disable_max_rows()

# Load your interruptions data
interruptions_2010 = pd.read_csv('/Users/shirleyaraizasantaella/Documents/UMich/Grad School Year 3/SI 649 Info Visualization/final project/SI649---Final-Project-Lizett-Shirley-Humaira/2010_service_interruptions.csv')  # Replace with your file path

# Preprocess ZIP codes to extract the first 5 digits
interruptions_2010['Zip'] = interruptions_2010['Zip'].str.split('-').str[0]

# Fetch GeoJSON data for Michigan ZIP codes
geojson_url = "https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/mi_michigan_zip_codes_geo.min.json"
response = requests.get(geojson_url)
if response.status_code == 200:
    geojson = response.json()
else:
    raise Exception("Failed to fetch GeoJSON data.")

# Dynamically filter GeoJSON to match ZIP codes in interruptions data
detroit_zip_codes = interruptions_2010['Zip'].unique()
geojson['features'] = [
    feature for feature in geojson['features']
    if feature['properties']['ZCTA5CE10'] in detroit_zip_codes
]

# Aggregate interruptions data by ZIP code
interruptions_agg = interruptions_2010.groupby('Zip').size().reset_index(name='Shutoff_Count')

# Create the Altair data source for GeoJSON
geo_data = alt.Data(values=geojson['features'])

# Choropleth Map
map_chart = alt.Chart(geo_data).mark_geoshape().transform_lookup(
    lookup='properties.ZCTA5CE10',  # Match the GeoJSON ZIP code property
    from_=alt.LookupData(interruptions_agg, 'Zip', ['Shutoff_Count'])  # Match ZIP codes in your data
).encode(
    color=alt.Color('Shutoff_Count:Q', title='Water Shutoffs', scale=alt.Scale(scheme='reds')),
    tooltip=['properties.ZCTA5CE10:N', 'Shutoff_Count:Q']
).project(
    type='albersUsa'
).properties(
    width=800,
    height=600,
    title="Water Shutoffs by ZIP Code in Detroit"
).configure_title(
    fontSize=16,
    anchor='start'
).configure_legend(
    titleFontSize=12,
    labelFontSize=10
).interactive()  # Enable zooming and panning

# Display the map
map_chart


/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

### Monthly Patterns of Shutoffs

In [5]:
# Ensure the WO_CREATEDATE column is in datetime format
interruptions_2010['WO_CREATEDATE'] = pd.to_datetime(interruptions_2010['WO_CREATEDATE'], errors='coerce')

# Extract month and year
interruptions_2010['Month'] = interruptions_2010['WO_CREATEDATE'].dt.month
interruptions_2010['Year'] = interruptions_2010['WO_CREATEDATE'].dt.year

# Group data by year and month
shutoffs_by_month_year = interruptions_2010.groupby(['Year', 'Month']).size().reset_index(name='Shutoff_Count')

# Create the heatmap
heatmap = alt.Chart(shutoffs_by_month_year).mark_rect().encode(
    x=alt.X('Month:O', title='Month'),
    y=alt.Y('Year:O', title='Year'),
    color=alt.Color('Shutoff_Count:Q', title='Shutoff Count'),
    tooltip=['Year:O', 'Month:O', 'Shutoff_Count:Q']
).properties(
    title='Monthly Patterns of Water Shutoffs',
    width=600,
    height=400
)

heatmap




/var/folders/6t/t91z119x51q_lpb_lspp0sjw0000gn/T/ipykernel_74577/3108252407.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  interruptions_2010['WO_CREATEDATE'] = pd.to_datetime(interruptions_2010['WO_CREATEDATE'], errors='coerce')


alt.Chart(...)

In [6]:
shutoff_reasons = interruptions_2010['WO_DESC'].value_counts().reset_index()
shutoff_reasons.columns = ['Reason', 'Count']

# Create the bar chart
alt.Chart(shutoff_reasons).mark_bar().encode(
    x=alt.X('Count:Q', title='Number of Shutoffs'),
    y=alt.Y('Reason:N', sort='-x', title='Reason'),
    tooltip=['Reason:N', 'Count:Q']
).properties(
    title='Reasons for Water Shutoffs',
    width=600,
    height=400
)

/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

### Cumulative changes over time

In [7]:
shutoffs_cumulative = shutoffs_over_time.copy()
shutoffs_cumulative['Cumulative_Count'] = shutoffs_cumulative['Shutoff_Count'].cumsum()

# Create the area chart
alt.Chart(shutoffs_cumulative).mark_area().encode(
    x=alt.X('YearMonth:T', title='Year-Month'),
    y=alt.Y('Cumulative_Count:Q', title='Cumulative Number of Shutoffs'),
    tooltip=['YearMonth:T', 'Cumulative_Count:Q']
).properties(
    title='Cumulative Water Shutoffs Over Time',
    width=600,
    height=400
)

alt.Chart(...)

### Trend of water shutoffs using all seven years worth of data

In [8]:
import pandas as pd
import altair as alt

# Ensure Altair's row limit is disabled
alt.data_transformers.disable_max_rows()

# Combine all CSV files into one DataFrame (example file paths used here)
file_paths = [
    '/Users/shirleyaraizasantaella/Documents/UMich/Grad School Year 3/SI 649 Info Visualization/final project/SI649---Final-Project-Lizett-Shirley-Humaira/2010_service_interruptions.csv',
    '/Users/shirleyaraizasantaella/Documents/UMich/Grad School Year 3/SI 649 Info Visualization/final project/SI649---Final-Project-Lizett-Shirley-Humaira/2011_service_interruptions.csv',
    '/Users/shirleyaraizasantaella/Documents/UMich/Grad School Year 3/SI 649 Info Visualization/final project/SI649---Final-Project-Lizett-Shirley-Humaira/2012_service_interruptions.csv',
    '/Users/shirleyaraizasantaella/Documents/UMich/Grad School Year 3/SI 649 Info Visualization/final project/SI649---Final-Project-Lizett-Shirley-Humaira/2013_service_interruptions.csv',
    '/Users/shirleyaraizasantaella/Documents/UMich/Grad School Year 3/SI 649 Info Visualization/final project/SI649---Final-Project-Lizett-Shirley-Humaira/2014_service_interruptions.csv',
    '/Users/shirleyaraizasantaella/Documents/UMich/Grad School Year 3/SI 649 Info Visualization/final project/SI649---Final-Project-Lizett-Shirley-Humaira/2015_service_interruptions.csv',
    '/Users/shirleyaraizasantaella/Documents/UMich/Grad School Year 3/SI 649 Info Visualization/final project/SI649---Final-Project-Lizett-Shirley-Humaira/2016_service_interruptions.csv',
    '/Users/shirleyaraizasantaella/Documents/UMich/Grad School Year 3/SI 649 Info Visualization/final project/SI649---Final-Project-Lizett-Shirley-Humaira/2017_service_interruptions.csv'
]
all_data = pd.concat([pd.read_csv(file) for file in file_paths])

# Ensure the WO_CREATEDATE column is in datetime format
all_data['WO_CREATEDATE'] = pd.to_datetime(all_data['WO_CREATEDATE'], errors='coerce')

# Filter out rows with invalid dates
all_data = all_data.dropna(subset=['WO_CREATEDATE'])

# Create a YearMonth column for grouping
all_data['YearMonth'] = all_data['WO_CREATEDATE'].dt.to_period('M').dt.to_timestamp()

# Group data by YearMonth
shutoffs_over_time = all_data.groupby('YearMonth').size().reset_index(name='Shutoff_Count')

# Define a selection for the slider
time_slider = alt.selection_interval(encodings=['x'])

# Create the base chart
base_chart = alt.Chart(shutoffs_over_time).mark_line().encode(
    x=alt.X('YearMonth:T', title='Year-Month'),
    y=alt.Y('Shutoff_Count:Q', title='Number of Shutoffs'),
    tooltip=['YearMonth:T', 'Shutoff_Count:Q']
).add_params(
    time_slider
).properties(
    title='Trend of Water Shutoffs Over Time',
    width=800,
    height=400
)

# Add a zoomed-in view
zoom_chart = alt.Chart(shutoffs_over_time).mark_line().encode(
    x=alt.X('YearMonth:T', title='Year-Month'),
    y=alt.Y('Shutoff_Count:Q', title='Number of Shutoffs'),
    tooltip=['YearMonth:T', 'Shutoff_Count:Q']
).transform_filter(
    time_slider
).properties(
    title='Zoomed-in View of Selected Time Range',
    width=800,
    height=400
)

# Combine the charts
interactive_chart = base_chart & zoom_chart
interactive_chart


/var/folders/6t/t91z119x51q_lpb_lspp0sjw0000gn/T/ipykernel_74577/457286734.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  all_data['WO_CREATEDATE'] = pd.to_datetime(all_data['WO_CREATEDATE'], errors='coerce')


alt.VConcatChart(...)

### Geographic Distribution of Shutoffs over seven years

In [9]:
print(all_data.head())

   WO_NUMBER WO_CREATEDATE WO_COMPLETEDATE   WO_DESC  WO_TYPE  \
0      13195    2010-01-12         1/12/10      VOID      318   
1      13196    2010-01-12         1/12/10      VOID      318   
2        429    2010-01-05          1/6/10  COMPLETE      324   
3      13197    2010-01-12         1/12/10      VOID      318   
4      13198    2010-01-12         1/12/10      VOID      318   

               WO_TYPEDESC  WO_CUSTOMERNUMBER       SALES_CLASS  \
0  CC TURN-OFF NON-PAYMENT          100008301   CITY COMMERCIAL   
1  CC TURN-OFF NON-PAYMENT          100009301   CITY COMMERCIAL   
2  CC NON-RESIDENTIAL SHUT         8503866300  CITY RESIDENTIAL   
3  CC TURN-OFF NON-PAYMENT          100011301   CITY COMMERCIAL   
4  CC TURN-OFF NON-PAYMENT          100012301   CITY COMMERCIAL   

  SERVICEADDRESSSTREETNUMBER ServiceAddressStreetName  Year  Month  \
0                       40XX          E JEFFERSON AVE  2010      1   
1                       40XX          E JEFFERSON AVE  2010      1

In [15]:
import pandas as pd
import altair as alt
import requests
import json

# Ensure Altair's row limit is disabled
alt.data_transformers.disable_max_rows()

# List of file paths for the CSV files
file_paths = [
    '/Users/shirleyaraizasantaella/Documents/UMich/Grad School Year 3/SI 649 Info Visualization/final project/SI649---Final-Project-Lizett-Shirley-Humaira/2010_service_interruptions.csv',
    '/Users/shirleyaraizasantaella/Documents/UMich/Grad School Year 3/SI 649 Info Visualization/final project/SI649---Final-Project-Lizett-Shirley-Humaira/2011_service_interruptions.csv',
    '/Users/shirleyaraizasantaella/Documents/UMich/Grad School Year 3/SI 649 Info Visualization/final project/SI649---Final-Project-Lizett-Shirley-Humaira/2012_service_interruptions.csv',
    '/Users/shirleyaraizasantaella/Documents/UMich/Grad School Year 3/SI 649 Info Visualization/final project/SI649---Final-Project-Lizett-Shirley-Humaira/2013_service_interruptions.csv',
    '/Users/shirleyaraizasantaella/Documents/UMich/Grad School Year 3/SI 649 Info Visualization/final project/SI649---Final-Project-Lizett-Shirley-Humaira/2014_service_interruptions.csv',
    '/Users/shirleyaraizasantaella/Documents/UMich/Grad School Year 3/SI 649 Info Visualization/final project/SI649---Final-Project-Lizett-Shirley-Humaira/2015_service_interruptions.csv',
    '/Users/shirleyaraizasantaella/Documents/UMich/Grad School Year 3/SI 649 Info Visualization/final project/SI649---Final-Project-Lizett-Shirley-Humaira/2016_service_interruptions.csv',
    '/Users/shirleyaraizasantaella/Documents/UMich/Grad School Year 3/SI 649 Info Visualization/final project/SI649---Final-Project-Lizett-Shirley-Humaira/2017_service_interruptions.csv'
]


# Load and preprocess data from all CSV files
all_data = []
for file in file_paths:
    data = pd.read_csv(file)
    
    # Ensure 'Zip' is a string and handle missing values
    data['Zip'] = data['Zip'].astype(str)
    data = data.dropna(subset=['Zip'])  # Remove rows with missing ZIP codes
    
    # Extract the first 5 digits of ZIP codes (ZIP+4 to standard 5-digit format)
    data['Zip'] = data['Zip'].str.split('-').str[0].str.zfill(5)  # Ensure all ZIP codes are 5 digits
    
    # Parse dates and extract Year and Month
    data['WO_CREATEDATE'] = pd.to_datetime(data['WO_CREATEDATE'], errors='coerce')
    data['WO_COMPLETEDATE'] = pd.to_datetime(data['WO_COMPLETEDATE'], errors='coerce')
    data['Year'] = data['WO_CREATEDATE'].dt.year
    data['Month'] = data['WO_CREATEDATE'].dt.month
    
    # Append to the list
    all_data.append(data)

# Combine all data into a single DataFrame
interruptions_data = pd.concat(all_data)

# Proceed with further processing on the combined DataFrame
interruptions_data['Zip'] = interruptions_data['Zip'].str.split('-').str[0].str.zfill(5)  # Ensure all ZIP codes are 5 digits
interruptions_data = interruptions_data[interruptions_data['Zip'] != '49238']


/var/folders/6t/t91z119x51q_lpb_lspp0sjw0000gn/T/ipykernel_74577/3980320092.py:35: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['WO_CREATEDATE'] = pd.to_datetime(data['WO_CREATEDATE'], errors='coerce')
/var/folders/6t/t91z119x51q_lpb_lspp0sjw0000gn/T/ipykernel_74577/3980320092.py:36: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['WO_COMPLETEDATE'] = pd.to_datetime(data['WO_COMPLETEDATE'], errors='coerce')
/var/folders/6t/t91z119x51q_lpb_lspp0sjw0000gn/T/ipykernel_74577/3980320092.py:35: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['WO_CREATEDATE'] = pd.to_d

In [28]:
# Fetch GeoJSON data for Michigan ZIP codes
geojson_url = "https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/mi_michigan_zip_codes_geo.min.json"
response = requests.get(geojson_url)
if response.status_code == 200:
    geojson = response.json()
else:
    raise Exception("Failed to fetch GeoJSON data.")

# Dynamically filter GeoJSON to match ZIP codes in interruptions data
detroit_zips = set(feature['properties']['ZCTA5CE10'] for feature in geojson['features'])
interruptions_data = interruptions_data[interruptions_data['Zip'].isin(detroit_zips)]

# Aggregate interruptions data by ZIP code and year
interruptions_agg = interruptions_data.groupby(['Zip', 'Year']).size().reset_index(name='Shutoff_Count')

# Ensure the Year field is an integer
interruptions_agg['Year'] = interruptions_agg['Year'].astype(int)

# Create the Altair data source for GeoJSON
geo_data = alt.Data(values=geojson['features'])

# Create dropdown selectors for Year and Zip
zip_selector = alt.binding_select(options=['All'] + sorted(interruptions_agg['Zip'].unique()), name='ZIP Code:')

year_selector = alt.binding_range(min=interruptions_agg['Year'].min(), max=interruptions_agg['Year'].max(), step=1)
year_filter = alt.selection_point(fields=['Year'], bind=year_selector, init={'Year': interruptions_agg['Year'].min()})

# Add Altair selection for filters
zip_filter = alt.selection_point(fields=['Zip'], bind=zip_selector, name='zip_filter')

# Ensure filters can handle the "All" option for Zip
zip_filter_condition = alt.datum.Zip if 'All' not in zip_filter else True

# Choropleth Map
map_chart = alt.Chart(geo_data).mark_geoshape().transform_lookup(
    lookup='properties.ZCTA5CE10',  # Match the GeoJSON ZIP code property
    from_=alt.LookupData(interruptions_agg, 'Zip', ['Year', 'Shutoff_Count'])  # Match ZIP codes in your data
).encode(
    color=alt.Color('Shutoff_Count:Q', title='Water Shutoffs', scale=alt.Scale(scheme='reds')),
    tooltip=['properties.ZCTA5CE10:N', 'Shutoff_Count:Q']  # Simplified tooltip
).transform_filter(
    year_filter  # Filter by selected year
).transform_filter(
    zip_filter  # Filter by selected ZIP
).project(
    type='albersUsa'
).properties(
    width=800,
    height=600,
    title="Water Shutoffs by ZIP Code and Year in Detroit"
).configure_title(
    fontSize=16,
    anchor='start'
).configure_legend(
    titleFontSize=12,
    labelFontSize=10
).add_params(
    year_filter,
    zip_filter
)

# Display the map
map_chart


/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/altair/vegalite/v5/api.py:381: AltairDeprecationWarning: Use 'value' instead of 'init'.
  warnings.warn(


TypeError: altair.vegalite.v5.schema.core.SelectionParameter() got multiple values for keyword argument 'value'

In [17]:
print(interruptions_agg['Year'].unique())
print(interruptions_agg['Zip'].unique())

[2012 2015 2016 2017 2010 2011 2013 2014]
['48127' '48201' '48202' '48203' '48204' '48205' '48206' '48207' '48208'
 '48209' '48210' '48211' '48212' '48213' '48214' '48215' '48216' '48217'
 '48219' '48220' '48221' '48223' '48224' '48225' '48226' '48227' '48228'
 '48234' '48235' '48236' '48238' '48239' '48240' '48243']


In [18]:
year_selector = alt.binding_select(options=sorted(interruptions_agg['Year'].unique()), name='Year:')
zip_selector = alt.binding_select(options=['All'] + sorted(interruptions_agg['Zip'].unique()), name='ZIP Code:')

In [26]:
debug_chart = alt.Chart(interruptions_agg).mark_bar().encode(
    x='Year:O',
    y='Shutoff_Count:Q',
    color=alt.condition(year_filter, alt.value('steelblue'), alt.value('lightgray'))
).add_params(year_filter)

debug_chart

/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)